# The Delta Method

## Load Packages and Extra Functions

The notebook first implements the delta method step-by-step. At the end it also presents a the function `DeltaMethod()` from the (local) `FinEcmt_OLS` module that wraps those calculations.

In [1]:
MyModulePath = joinpath(pwd(),"jlFiles")
!in(MyModulePath,LOAD_PATH) && push!(LOAD_PATH,MyModulePath);

In [2]:
using FinEcmt_OLS, DelimitedFiles, Statistics

## Load Data

In [3]:
x  = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)
x  = x[:,2]              #x is an excess return in % (on the US equity market)
T  = size(x,1)

388

## Point Estimates of the Mean and Variance

In [4]:
μ   = mean(x)                  #estimates of the mean and variance
σ²  = var(x,corrected=false)

printblue("mean and variance:")
momNames = ["μ","σ²"]
printmat([μ,σ²];rowNames=momNames)

mean and variance:
μ      0.602
σ²    21.142



The variance-covariance matrix (called $V$) of the point estimates depends on the distribution of the data. With a normal distribution, the form is particularly simple. We use that approximation in the next cell. Another approach is to estimate $V$ from the moment conditions (see GMM).

In [5]:
V = [σ² 0;                 #variance-covariance matrix of the estimates of [μ,σ²]
     0  2*abs2(σ²)]/T

printmat(V;rowNames=momNames,colNames=momNames)

           μ        σ²
μ      0.054     0.000
σ²     0.000     2.304



## The Sharpe Ratio and Its Derivatives

The Sharpe ratio and its derivatives (with respect to the parameters of the
Sharpe ratio) are

$
SR  =\frac{\mu}{\sigma},\: \text{where}\: \beta=(\mu,\sigma^2)
$

Let $f(\beta)$ denote the Sharpe ratio where $\beta$ is a vector of parameters 
consisting of the mean and the variance ($\mu,\sigma^2$). The derivatives are then

$
\frac{\partial f(\beta)}{\partial\beta^{\prime}}  
= \begin{bmatrix}
\frac{1}{\sigma} & \frac{-\mu}{2 \sigma^3}
\end{bmatrix}
$

We will refer to the matrix of derivatives as $P$.

In [6]:
"""
    SRFn(μ,σ²)

Calculate the Sharpe ratio from the mean μ and variance σ²

"""
function SRFn(μ,σ²)
  σ  = sqrt(σ²)
  SR = μ/σ
  P = hcat(1/σ, -μ/(2*σ^3))     #Jacobian of SR, 1x2
  return SR, P
end

SRFn

In [7]:
(SR,P) = SRFn(μ,σ²)

printlnPs("Sharpe ratio: ",SR)

printblue("\nDerivatives of Sharpe ratio function wrt:")
printmat(P,colNames=momNames)

Sharpe ratio:      0.131

Derivatives of Sharpe ratio function wrt:
         μ        σ²
     0.217    -0.003



## Applying the Delta Method


Recall that if

$
\hat{\beta} \sim N(\beta_{0},V),
$

then the distribution of the function $f(\hat{\beta})$ is asymptotically

$
f(\hat{\beta}) \sim N(f(\beta_{0}),PVP')  
$

where $P$ are the derivatives of $f(\beta)$.

In [8]:
Std_SR = sqrt(only(P*V*P'))  #only() to convert from 1x1 matrix to scalar
tstat = SR/Std_SR

printblue("Results from the delta method:")
printmat([SR Std_SR tstat],colNames=["SR","Std(SR)","t-stat"])

Results from the delta method:
        SR   Std(SR)    t-stat
     0.131     0.051     2.567



## A Function for the Delta Method (extra)

is included below. It uses numerical derivatives from the `FiniteDiff.jl` package. 

To use this, first write a function that takes `(β,x)` as inputs (see `SRFn2(β,x)` below), where `β` is a vector of the parameters and `x` any data needed (for the Sharpe ratio, no data is needed). 

In [9]:
display("text/markdown",@doc DeltaMethod)

```
DeltaMethod(fn::Function,β,V,x=NaN)
```

Apply the delta method on the function `fn(β,x)`

# Input

  * `fn::Function`:     of the type fn(β,x)
  * `β::Vector`:        with parameters
  * `V::Matrix`:        Cov(β)
  * `x::VecOrMat`:      data (if any is needed)

### Requires

  * `using FiniteDiff: finite_difference_jacobian as jacobian`


In [10]:
using CodeTracking
println(@code_string  DeltaMethod(cos,[1],[1]))

function DeltaMethod(fn::Function,β,V,x=NaN)
    P = jacobian(b->fn(b,x),β)        #numerical Jacobian
    Cov_fn = P*V*P'
    return Cov_fn
end


In [11]:
"""
    SRFn2(β,x)

Function for Sharpe ratio in terms of the vector β. No derivatives
"""
function SRFn2(β,x=NaN)
  (μ,σ²) = β
  σ  = sqrt(σ²)
  SR = μ/σ
  return SR
end;

In [12]:
Var_SR = DeltaMethod(SRFn2,[μ,σ²],V)

printblue("Std of SR from DeltaMethod():")
printmat(sqrt(only(Var_SR)))

Std of SR from DeltaMethod():
     0.051

